# Compute final score

In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
df = pd.read_csv('./data/stats.txt', sep='<>| ', header=None, skiprows=1,
                 names=['sub', 'rel', 'obj', 'rank', 'stat', 'f012',
                        'f0', 'f1', 'f2', 'f01', 'f02', 'f12'], engine='python')

In [3]:
deg = pd.read_csv('./data/concept_degree.txt', sep=' ', header=None, names=['cui', 'deg'])
cui2deg = dict(zip(deg['cui'], deg['deg']))

In [4]:
def lookup(cui):
    return cui2deg[cui]

df['sub_deg'] = df.apply(lambda row: lookup(row['sub']), axis=1)
df['obj_deg'] = df.apply(lambda row: lookup(row['obj']), axis=1)

In [5]:
names = ['stat', 'sub_deg', 'obj_deg']
x = df[names].values
min_max_scaler = preprocessing.MinMaxScaler()
x_norm = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_norm, columns=names, index = df.index)
df[['stat_norm', 'sub_deg_norm', 'obj_deg_norm']] = df_temp
df['score'] = df['stat_norm'] + df['sub_deg_norm'] + df['obj_deg_norm']

In [6]:
df.to_csv('./data/sub_rel_obj_freq_score.tsv', sep='\t', header=None, columns=['sub', 'rel', 'obj', 'f012', 'score'],
          index=False, float_format='%.15f')